In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.5 MB/s 
     |████████████████████████████████| 7.6 MB 46.4 MB/s 
     |████████████████████████████████| 182 kB 57.0 MB/s 


In [2]:
import transformers

In [4]:
from transformers import ElectraTokenizer



tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

In [5]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [6]:
tokenizer.tokenize("뷀렋. 안녕 오늘은 날씨가 참 좋네요")

['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요']

토크나이저 훈련

1. 어떤 훈련은 사용할지
2. vocab_size 결정 >> 작으면 메모리 이득, 크면 의미를 잘 표현할 수 있다
3.  corpus >> 통계적으로 분석, vocab

tokenizer에 vocab 수동으로 직접 추가

vocab 300 ~ 350 [UNK1], [UNK2], [UNK3] ...

In [7]:
tokenizer.vocab_size

35000

In [8]:
tokenizer

PreTrainedTokenizer(name_or_path='monologg/koelectra-base-v3-discriminator', vocab_size=35000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
tokenizer.convert_tokens_to_ids(['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요'])

[3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [19]:
tokenizer.convert_ids_to_tokens([0,1,2,3, 155,156, 30000, 30001, 350001])

['<s>', '<pad>', '</s>', '<unk>', 'Ġ3', 'Ġmade', 'ĠJOHN', 'ĠReact', None]

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [21]:
tokenizer.tokenize('hello, world!')

['hello', ',', 'world', '!']

In [22]:
tokenizer.tokenize('안녕하세요')

['ᄋ',
 '##ᅡ',
 '##ᆫ',
 '##ᄂ',
 '##ᅧ',
 '##ᆼ',
 '##ᄒ',
 '##ᅡ',
 '##ᄉ',
 '##ᅦ',
 '##ᄋ',
 '##ᅭ']

In [11]:
tokenizer.vocab_size

30522

In [12]:
multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [13]:
multi_tokenizer.tokenize('안녕하세요')

['안', '##녕', '##하', '##세', '##요']

multi : 안녕 << vocab에 포함이 안되어있음


eletra : '안녕'이 vocab에 포함돼있음

In [14]:
multi_tokenizer.vocab_size

119547

In [15]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [16]:
tokenizer.convert_ids_to_tokens([111,22,33])

['Ġ-', 'Ġ"', 'Ġhave']

In [17]:
tokenizer.convert_tokens_to_ids('hello')

42891

'Webtest라고 불리는 [MASK] webpage로 외부적인 감독 없이 QA, 기계 번역, 문맥읽기, 요약 같은 걸 하도록 만들었다. 55F1, CoQA dataset을 QA했을 때 성능이 3/4 (SOTA)이다. (training exapmles없이)'

input = ' ~~~ [MASK] ~~~'

output = '원래문장'

라벨링 데이터를 >> Pretraining
오늘 나리가 너무 좋다 // 긍정

      '오늘 날씨가 너무 좋다'는 [긍정/부정] 문장이다. 정답 : 긍정

** 3가지 종류 언어모델 **

>> 언어에 대한 통계적 이해를 가진 모델

1. Decoder-only 모델 (GPT1, GPT2, GPT3, ..)
  - few shot learing 사용
  - 단점 : 모델이 너무 큼
  - 점점 더 모델이 커지고 있음(모델이 크고, 데이터를 많이 넣을수록 성능이 좋음)


2. Encoder-only 모델 Auto encoder (BERT, RoBERTa, Electra, Albert, ...)
  - input을 벡터로 바꿔준다
  - 벡터 의미와 관련이 있다.


  - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
  - Downstream task(번역, 문장 분류, 질의응답, ...)
  - Why?
    1. 성능이 기존 방식보다 훨씬 좋다
    2. 소량의 데이터로도 학습시간이 훨씬 빨리된다.

3. Encoder-Decoder 모델 >> Transformer 모델과 구조는 동일 (BART, T5, ...)
  - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
  - 생성 관련 Task 사용


encoder 모델은 생성 task 불가능, 추출은 가능
Decoder 모델 가능

챗봇을 만들고 싶다.
시나리오 1. Encoder 모델로 챗봇을 만드려면..?
>> (답변 1, 답변 2, 답변 3, ..)
질문 > 뽑아낸다

시나리오 2. Decoder 모델을 사용 > 좀 더 사람같은 느낌, 위험성, 엉뚱한 대답이 나올 때가 있음

embedding : 단어를 벡터로 바꿔주는 레이어(훈련 o)

Attention 알고리즘은 Query, key, value 3가지가 필요함
  - Q = Query : t 시점의 디코더 셀에서의 은닉 상태
  - k = Key : 모든 시점의 인코더 셀의 은닉 상태들
  - V = Value : 모든 시점의 인코더 셀의 은닉 상태들

[CLS]는 문장 전체를 나타내는 토큰


<질의응답>
[CLS] 포메라니안은 어떤걸 좋아해? [SEP] 포메라니안은 독일 태생의 사냥견으로, 노는걸 좋아한다. [SEP]


text2image 모델


BERT >> 문장을 벡터로 만들어준다

>> 벡터를 이미지로 만든다